In [1]:
pip install fastapi


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-multipart


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from threading import Thread

In [4]:
from fastapi import FastAPI, Request, Form, Query
from fastapi.templating import Jinja2Templates
from fastapi.responses import RedirectResponse, JSONResponse, HTMLResponse, FileResponse
from datetime import datetime, timedelta
from typing import List, Optional
import json, random, time
import glob
import os

In [5]:
import logging

# 로그 설정
logging.basicConfig(
    filename="server.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    encoding="utf-8"
)
log = logging.getLogger(__name__)


def safe_load_json(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)

    except json.JSONDecodeError as e:
        error_line = e.lineno
        print(f"❗ JSONDecodeError in file {filepath}: {e}")

        # 에러 발생 지점 주변 라인 출력
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()

            # 앞뒤로 3줄씩 출력
            start = max(0, error_line - 4)  # 0부터 시작이니 4를 빼야 3줄 앞부터 나옴
            end = min(len(lines), error_line + 3)

            print(f"\n🚩 Error 주변 라인 출력 ({start+1}~{end}):")
            for idx in range(start, end):
                line_content = lines[idx].rstrip("\n")
                line_indicator = "➡️" if (idx + 1) == error_line else "  "
                print(f"{line_indicator} Line {idx + 1}: {line_content}")

        return None

    except Exception as e:
        print(f"❗ Error reading file {filepath}: {e}")
        return None

In [6]:
from contextlib import asynccontextmanager

In [7]:
from fastapi.staticfiles import StaticFiles # Css file 설정


In [8]:
# status_info 전역 변수 선언
status_info = {}

# FAST API 시작 부분 - app start 대체
@asynccontextmanager
async def lifespan(app: FastAPI):
    try:
        with open("tickers.json", "r") as f:
            tickers_data = json.load(f)
    except FileNotFoundError:
        tickers_data = []

    for ex in tickers_data:
        #ex = entry["exchange"]
        for ticker in tickers_data.get(ex, []):
            print("📌 Exchange:", ex, "Ticker:", ticker)
        
        if ex not in status_info:
            status_info[ex] = {
                "status": "Running",
                "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
    # 앱 시작 시
    log.info("✅ App started")
    log.info("✅ Starting status_updater thread")
            
    thread = Thread(target=status_updater, daemon=True)
    thread.start()

    yield  # startup 이후 본격 앱 실행

    # 앱 종료
    log.info("🔴 App shutting down...")

# FastAPI 앱 선언 시 lifespan 추가
app = FastAPI(lifespan=lifespan)
app.mount("/static", StaticFiles(directory="static"), name="static")

In [9]:
@app.get("/favicon.ico")
async def favicon():
    return FileResponse("static/favicon.ico")

In [10]:
# 삭제가 안되는 default ticker 지정
default_tickers = {
    "Upbit": ["BTC/KRW"],
    "Korbit": ["BTC/KRW"],
    "Bithumb": ["BTC/KRW"],
    "Coinone": ["BTC/KRW"]
}

# 거래소와 티커를 정의
tickers_by_exchange = {
    "Upbit": ["BTC/KRW", "ETH/KRW"],
    "Bithumb": ["BTC/KRW", "ETH/KRW"],
    "Korbit": ["BTC/KRW"],
    "Coinone": ["BTC/KRW"]
}

# 에러 핸들러 등록
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    log.error(f"❗ Error occurred: {exc}")
    # 예외 발생 시 자동으로 로그 남기고, 사용자에게는 500 응답
    return JSONResponse(status_code=500, content={"detail": "Internal Server Error"})

# 서버 상태 확인
def status_updater():
    """Background thread function to update exchange statuses and timestamps."""
    while True:
        for ex, info in list(status_info.items()):
            if info["status"] == "Running":
                # Update last-updated timestamp for running collectors
                info["last_updated"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            # Randomly simulate an error state (~10% chance each cycle)
            if random.random() < 0.1:
                info["status"] = "Error" if info["status"] == "Running" else "Running"
        time.sleep(5)  # wait 5 seconds before next update cycle

In [11]:
# KST Seoul 기준 D-1일
try:
    from zoneinfo import ZoneInfo
    KST = ZoneInfo("Asia/Seoul")
except Exception:
    from datetime import timezone, timedelta as td
    KST = timezone(td(hours=9))

def get_previous_day_str():
    """Return yesterday's date as YYYYMMDD in KST."""
    now_kst = datetime.now(tz=KST)
    yesterday = now_kst - timedelta(days=1)
    return yesterday.strftime("%Y%m%d")


In [12]:
templates = Jinja2Templates(directory="templates")

# 모든 html 파일은 templetes에 있음
# templates/index.html


@app.get("/")
async def main_page(request: Request):
    # Load tickers from JSON
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    # Sort exchanges and tickers for consistent display order (optional)
    sorted_tickers = {ex: sorted(tickers_data[ex]) for ex in sorted(tickers_data)}
    context = {
        "request": request,
        "tickers_data": sorted_tickers,
        "selected_tickers": [],        # no tickers selected by default
        "result_data": None,           # no results to display on initial load
        "date_str": get_previous_day_str(),  # previous day date for reference
        "default_tickers": default_tickers
    }
    return templates.TemplateResponse("index.html", context)

@app.post("/add_ticker")
async def add_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and add the new one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
        if tickers_data is None:
            context["error"] = f"데이터 파일 형식에 문제가 있습니다: {file_path}"
            return templates.TemplateResponse("orderbook_select.html", context)
        
    symbol = ticker.strip().upper()  # normalize input symbol to uppercase
    if symbol and exchange:
        # Ensure the exchange key exists
        if exchange not in tickers_data:
            tickers_data[exchange] = []
        # Avoid duplicates
        if symbol not in tickers_data[exchange]:
            tickers_data[exchange].append(symbol)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    # Redirect back to main page
    return RedirectResponse(url="/", status_code=303)

@app.post("/delete_ticker")
async def delete_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and remove the specified one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
        
    # 기본 티커는 삭제 불가
    if exchange in default_tickers and ticker in default_tickers[exchange]:
        return RedirectResponse(url="/", status_code=303)
    
    
    if exchange in tickers_data and ticker in tickers_data[exchange]:
        tickers_data[exchange].remove(ticker)
        # If list becomes empty, optionally remove the exchange key
        if not tickers_data[exchange]:
            tickers_data.pop(exchange)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    return RedirectResponse(url="/", status_code=303)


In [13]:
@app.get("/orderbook_list")
async def show_orderbook_data(
    request: Request,
    exchange: str = Query(None),
    ticker: str   = Query(None)
):
    
    if not ticker:
        return templates.TemplateResponse("orderbook_list.html", {
            "request": request,
            "exchange": exchange,
            "ticker": None,
            "result_data": None,
            "date_str": get_previous_day_str(),
            "default_tickers": default_tickers,
            "error_msg": "티커를 하나 이상 선택해주세요."
        })

    # 공통 함수 재사용
    date_str = get_previous_day_str()
    result_data, selected_tickers = load_orderbook_data(ticker, date_str)
    
    return templates.TemplateResponse("orderbook_list.html", {
        "request": request,
        "exchange": exchange,
        "ticker": ticker,
        "result_data": result_data,
        "date_str": date_str,
        "default_tickers": default_tickers
    })

In [14]:
def load_orderbook_data(ticker: str, date_str: str):
    result_data = {}
    selected_by_exchange = {}

    if ":" in ticker:
        exchange, symbol = ticker.split(":", 1)
        ticker_key = f"{exchange} - {symbol}"
        result_data[ticker_key] = []
        pattern = f"orderbook_data/{date_str}/{exchange.lower()}_orderbook_{date_str}_*.json"
        for filename in sorted(glob.glob(pattern)):
            try:
                with open(filename, "r", encoding="utf-8") as f:
                    entries = json.load(f)
                for data in entries:
                    if not isinstance(data, dict):
                        continue
                    ts_ms = int(data.get("timestamp", 0))
                    dt = datetime.fromtimestamp(ts_ms / 1000.0, tz=KST)
                    time_str = dt.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3] + " KST"

                    # 호가 테이블용 양식 맞추기 (bid/ask 각각 첫 10개 정리)
                    top_bids = data.get("bids", [])[:10]
                    top_asks = data.get("asks", [])[:10]
                    rows = []
                    for i in range(10):
                        bid = top_bids[i] if i < len(top_bids) else ["", ""]
                        ask = top_asks[i] if i < len(top_asks) else ["", ""]
                        rows.append({
                            "bid_price": bid[0],
                            "bid_qty": bid[1],
                            "ask_price": ask[0],
                            "ask_qty": ask[1]
                        })

                    snapshot = {
                        "timestamp": ts_ms,
                        "time": time_str,
                        "rows": rows
                    }
                    result_data[ticker_key].append(snapshot)
            except Exception as e:
                print(f"❗ Error reading {filename}: {e}")
                continue

    for ticker_key in result_data:
        result_data[ticker_key].sort(key=lambda x: x["timestamp"])

    return result_data, ticker


In [15]:
# ticker 관련 함수들
@app.post("/add_ticker")
async def add_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and add the new one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    symbol = ticker.strip().upper()  # normalize input symbol to uppercase
    if symbol and exchange:
        # Ensure the exchange key exists
        if exchange not in tickers_data:
            tickers_data[exchange] = []
        # Avoid duplicates
        if symbol not in tickers_data[exchange]:
            tickers_data[exchange].append(symbol)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    # Redirect back to main page
    return RedirectResponse(url="/", status_code=303)

@app.post("/delete_ticker")
async def delete_ticker(request: Request, exchange: str = Form(...), ticker: str = Form(...)):
    # Load current tickers and remove the specified one
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    if exchange in tickers_data and ticker in tickers_data[exchange]:
        tickers_data[exchange].remove(ticker)
        # If list becomes empty, optionally remove the exchange key
        if not tickers_data[exchange]:
            tickers_data.pop(exchange)
    # Save back to JSON
    with open("tickers.json", "w") as f:
        json.dump(tickers_data, f, indent=4)
    return RedirectResponse(url="/", status_code=303)

In [16]:
@app.get("/orderbook_panel")
async def orderbook_panel(
    request: Request,
    ticker: str = Query(...),
    date: Optional[str] = Query(None)
):
    
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    # Sort exchanges and tickers for consistent display order (optional)
    sorted_tickers = {ex: sorted(tickers_data[ex]) for ex in sorted(tickers_data)}
    
    date_str = date or get_previous_day_str()

    # ticker: "Upbit:BTC/KRW" → exchange, symbol 분리
    if ":" not in ticker:
        return HTMLResponse("❌ Invalid ticker format", status_code=400)
    exchange, symbol = ticker.split(":", 1)

    pattern = f"orderbook_data/{date_str}/{exchange.lower()}_orderbook_{date_str}_*.json"
    latest_entry = None
    latest_ts = 0
    latest_filename = ""

    for filename in sorted(glob.glob(pattern)):
        try:
            with open(filename, "r", encoding="utf-8") as f:
                entries = json.load(f)
            for data in entries:
                ts = int(data.get("timestamp", 0))
                if ts > latest_ts:
                    latest_ts = ts
                    latest_entry = data
                    latest_filename = filename
        except:
            continue

    if not latest_entry:
        return HTMLResponse("❗ 데이터가 없습니다.", status_code=404)

    # 시간 변환
    dt = datetime.fromtimestamp(int(latest_ts) / 1000.0, tz=KST)
    time_str = dt.strftime("%Y-%m-%d %H:%M:%S") + " KST"

    return templates.TemplateResponse("orderbook_panel.html", {
        "request": request,
        "ticker": f"{exchange} - {symbol}",
        "bids": latest_entry.get("bids", [])[:10],
        "asks": latest_entry.get("asks", [])[:10],
        "time_str": time_str,
        "filename": pattern,
    })


In [17]:
def get_available_hours(exchange: str, date: str) -> List[str]:
    """특정 날짜와 거래소에 대해 실제 존재하는 JSON 파일의 시간대 리스트를 반환"""
    path_pattern = f"orderbook_data/{date}/{exchange.lower()}_orderbook_{date}_*.json"
    files = glob.glob(path_pattern)
    
    available_hours = sorted([
        os.path.basename(f).split("_")[-1].split(".")[0]
        for f in files
    ])

    return available_hours

@app.get("/orderbook")
async def orderbook_view(
    request: Request,
    exchange: str = Query(None),
    ticker: str   = Query(None),
    date: str     = Query(None),
    hour: str     = Query(None),
    snapshot_index: Optional[int] = Query(None)
):
    
    if not exchange:
        exchange = list(tickers_by_exchange.keys())[0]
    
    # 거래소만 선택되고 티커가 없으면 첫번째 티커를 자동선택
    if exchange and not ticker:
        ticker = tickers_by_exchange[exchange][0]
    
    available_hours = get_available_hours(exchange, date or get_previous_day_str())
    
    context = {
        "request": request,
        "tickers": tickers_by_exchange,
        "selected": {
            "exchange": exchange,
            "ticker": ticker,
            "date": date or get_previous_day_str(),
            "hour": hour
        },
        "available_hours": available_hours,
        "snapshots": [],
    }

    if not (exchange and ticker and date and hour):
        return templates.TemplateResponse("orderbook_select.html", context)
    
    # If required params are missing, render the form with default date (yesterday)
    if not (exchange and ticker and date and hour):
        return templates.TemplateResponse("orderbook_select.html", context)
    
    # Build the JSON file path based on user selection
    filename = f"{exchange.lower()}_orderbook_{date}_{hour}.json"
    file_path = f"orderbook_data/{date}/{filename}"
    try:
        data = safe_load_json(file_path)
        if data is None:
            context["error"] = f"데이터 파일 형식에 문제가 있습니다: {file_path}"
            return templates.TemplateResponse("orderbook_select.html", context)
    except FileNotFoundError:
        # If file is not found, re-render form with an error message
        context = {
            "request": request,
            "tickers": tickers_by_exchange,
            "selected": {"exchange": exchange, "ticker": ticker, "date": date, "hour": hour},
            "snapshots": [],
            "date": date,
            "error": f"데이터 파일을 찾을 수 없습니다: {filename}"
        }
        return templates.TemplateResponse("orderbook_select.html", context)
    
    # Parse JSON content to extract snapshots
    snapshots = []
    for entry in data:
        if not isinstance(entry, dict):
            continue
        ts = entry.get("timestamp")  # timestamp in milliseconds
        # Convert timestamp to human-readable time string
        if ts:
            dt = datetime.fromtimestamp(int(ts) / 1000.0)  # convert ms to seconds
            time_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        else:
            time_str = "Unknown Time"
        # Get top 10 bids and asks (each entry like [price, quantity])
        top_bids = entry.get("bids", [])[:10]
        top_asks = entry.get("asks", [])[:10]
        # Prepare rows combining bid and ask for side-by-side display
        rows = []
        for i in range(10):
            # Fill with empty values if there are fewer than 10 bids or asks
            if i < len(top_bids):
                bid_price, bid_qty = top_bids[i]
            else:
                bid_price, bid_qty = ("", "")
            if i < len(top_asks):
                ask_price, ask_qty = top_asks[i]
            else:
                ask_price, ask_qty = ("", "")
            rows.append({
                "bid_price": bid_price,
                "bid_qty": bid_qty,
                "ask_price": ask_price,
                "ask_qty": ask_qty
            })
        snapshots.append({"time": time_str, "rows": rows})
    
    # Sort snapshots by time if not already sorted (optional)
    snapshots.sort(key=lambda x: x["time"])
    
    # Ensure the selected snapshot index is within range
    if snapshots:
        if snapshot_index is None or snapshot_index < 0 or snapshot_index >= len(snapshots):
            snapshot_index = 0  # 기본값
        selected_snapshot = snapshots[snapshot_index]
    else:
        selected_snapshot = None
    
    context = {
        "request": request,
        "tickers": tickers_by_exchange,
        "selected": {"exchange": exchange, "ticker": ticker, "date": date, "hour": hour},
        "snapshots": snapshots,
        "snapshot_index": snapshot_index,
        "selected_snapshot": selected_snapshot,
        "available_hours": available_hours,
    }
    return templates.TemplateResponse("orderbook_select.html", context)

In [18]:
# 서비스 검색 기능 추가
@app.post("/search_log")
async def search_log(
    request: Request,
    log_date: str = Form(...),
    log_exchange: str = Form(""),
    tickers: Optional[List[str]] = Form(None)
):
    log_results = []
    try:
        with open("server.log", "r", encoding="utf-8") as f:
            for line in f:
                if log_date in line and (not log_exchange or log_exchange.lower() in line.lower()):
                    log_results.append(line.strip())
    except FileNotFoundError:
        log_results.append("⚠️ server.log 파일이 없습니다.")

    # load tickers data
    with open("tickers.json", "r") as f:
        tickers_data = json.load(f)
    sorted_tickers = {ex: sorted(tickers_data[ex]) for ex in sorted(tickers_data)}

    # 공통 로직 재사용
    date_str = get_previous_day_str()
    result_data, selected_tickers = load_orderbook_data(tickers, date_str)

        # 페이징 처리
    page = int(request.query_params.get("page", 1))
    page_size = 10
    paged_result_data = {}
    pagination_meta = {}
    for key, snapshots in result_data.items():
        total_pages = max(1, (len(snapshots) + page_size - 1) // page_size)
        start = (page - 1) * page_size
        end = start + page_size
        paged_result_data[key] = snapshots[start:end]
        pagination_meta[key] = {
            "current_page": page,
            "total_pages": total_pages
        }
    
    return templates.TemplateResponse("index.html", {
        "request": request,
        "tickers_data": sorted_tickers,
        "selected_tickers": selected_tickers,
        "result_data": paged_result_data or None,
        "pagination_meta": pagination_meta,
        "date_str": date_str,
        "default_tickers": default_tickers,
        "log_results": log_results,
        "log_date": log_date,
        "log_exchange": log_exchange
    })
    return templates.TemplateResponse("index.html", context)


In [19]:
pip install nest_asyncio


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install uvicorn


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
# jupyter 실행 시 필요
import nest_asyncio
import uvicorn

In [ ]:
# jupyter 실행 
if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [15148]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


📌 Exchange: Upbit Ticker: BTC/KRW
📌 Exchange: Korbit Ticker: BTC/KRW
📌 Exchange: Bithumb Ticker: BTC/KRW
📌 Exchange: Coinone Ticker: BTC/KRW
INFO:     127.0.0.1:63858 - "GET / HTTP/1.1" 200 OK
❗ Error reading orderbook_data/20250406\bithumb_orderbook_20250406_10.json: Expecting value: line 2 column 1 (char 2)
❗ Error reading orderbook_data/20250406\bithumb_orderbook_20250406_11.json: Expecting value: line 2 column 1 (char 2)
INFO:     127.0.0.1:63858 - "GET /orderbook_list?ticker=Bithumb:BTC/KRW HTTP/1.1" 200 OK
❗ Error reading orderbook_data/20250406\coinone_orderbook_20250406_10.json: Expecting value: line 2 column 1 (char 2)
INFO:     127.0.0.1:63859 - "GET /orderbook_list?ticker=Coinone:BTC/KRW HTTP/1.1" 200 OK
❗ Error reading orderbook_data/20250406\bithumb_orderbook_20250406_11.json: Expecting value: line 2 column 1 (char 2)
INFO:     127.0.0.1:63870 - "GET /orderbook_list?ticker=Bithumb:BTC/KRW HTTP/1.1" 200 OK
INFO:     127.0.0.1:63897 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.